In [1]:
!pip install gradio


  You can safely remove it manually.



   ---------------------------------------- 0.0/59.7 MB ? eta -:--:--
   ------ --------------------------------- 10.0/59.7 MB 56.4 MB/s eta 0:00:01
   ----------------------- ---------------- 34.9/59.7 MB 88.5 MB/s eta 0:00:01
   ---------------------------------------  59.5/59.7 MB 99.7 MB/s eta 0:00:01
   ---------------------------------------- 59.7/59.7 MB 82.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 12.7/12.7 MB 89.1 MB/s eta 0:00:00
  Attempting uninstall: brotli
    Found existing installation: Brotli 1.0.9
    Uninstalling Brotli-1.0.9:
      Successfully uninstalled Brotli-1.0.9
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.11.1
    Uninstalling tomlkit-0.11.1:
      Successfully uninstalled tomlkit-0.11.1


In [18]:
import gradio as gr
import openai
import time

# 🔐 Chargement de la clé depuis un fichier .env de type OPENAI_API_KEY=sk-...
def charger_cle_api():
    try:
        with open("ENV/env.txt", "r") as f:
            lignes = f.readlines()
            for ligne in lignes:
                if ligne.startswith("OPENAI_API_KEY="):
                    return ligne.split("=", 1)[1].strip()
    except Exception as e:
        return None

# 🤖 Interaction complète avec l'assistant Lyra_candidatures
def repondre(prompt, mode_simulation):
    if mode_simulation:
        return "[Mode Simulation]\n" + prompt[::-1]

    cle_api = charger_cle_api()
    if not cle_api:
        return "❌ Clé API introuvable dans ENV/env.txt ou format incorrect."

    try:
        client = openai.OpenAI(api_key=cle_api)

        # Création dynamique du thread
        # 🔒 Réutilisation d'un thread existant
        thread_id = "thread_FlIBwDXn4HUAMbmTqtzOJvL9"

        # Envoi du message utilisateur
        client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=prompt
        )

        # Lancement du run
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id="asst_0jnNRiJzmqgjrHM0XSFmcLxK"
        )

        # Polling de l'état du run
        while True:
            run_status = client.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
            )
            if run_status.status == "completed":
                break
            elif run_status.status in ["failed", "cancelled", "expired"]:
                return f"⚠️ Le run a échoué : {run_status.status}"
            time.sleep(1)

        # Récupération du dernier message
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        dernier_message = messages.data[0].content[0].text.value
        return "✅ Réponse de Lyra :\n" + dernier_message

    except Exception as e:
        return f"❌ Erreur OpenAI : {str(e)}"

# 🎛️ Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("""### 🤖 Console RH Lyra (Assistant OpenAI)
Assistant intelligent simulant un service RH rapide, cohérent et personnalisé.
Utilise l'assistant OpenAI `Lyra_candidatures`.
""")

    prompt = gr.Textbox(label="✉️ Message utilisateur", placeholder="Entrez Ici votre message RH...", lines=4)
    mode_simulation = gr.Checkbox(label="✅ Mode simulation (réponse inversée)", value=False)
    bouton = gr.Button("📤 Envoyer")
    reponse = gr.Textbox(label="💬 Réponse de Lyra", lines=6)

    bouton.click(fn=repondre, inputs=[prompt, mode_simulation], outputs=reponse)

demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
